<a href="https://colab.research.google.com/github/stefantatur/AI-agent/blob/main/Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Инициализация и настройка окружения

In [1]:
import os, getpass
import requests
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

def _set_env(var: str):
    if not os.environ.get(var): # проверяет существует ли переменная окружения
        os.environ[var] = getpass.getpass(f"{var}: ") # если нет, создает новую

# стараемся не хардкодить и задавать все важные ключи в начале
_set_env("OPENAI_API_KEY") # используем set_env для безопасного чтения ключей
_set_env("LANGCHAIN_API_KEY") # Ключ для сервисов LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

ModuleNotFoundError: No module named 'langchain_openai'

 ### 2. Определение арифметических инструментов

In [ ]:
def multiply(a: int, b: int) -> int:
    """Умножает два числа"""
    return a * b

def add(a: int, b: int) -> int:
    """Складывает два числа"""
    return a + b

def divide(a: int, b: int) -> float:
    """Делит число a на число b"""
    if b == 0:
        raise ValueError("Делить на ноль нельзя!") # небольшая обработка исключений
    return a / b

tools = [add, multiply, divide] # наш функционал

### 3. Настройка языковой модели

In [ ]:
llm = ChatOpenAI(model="gpt-4o",
                 openai_proxy="http://43.153.39.191:13001",
                 temperature=0)

''' параметр temperature 0 установлен,
так как при конвертации валют,
очень важно получать четкие ответы '''

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False) # привязываем инструменты к llm-ки и отключаем параллельный вызов функций
sys_msg = SystemMessage(content="Вы помощник для арифметики и конвертации валют.") # задаем роль и поведение модели

### 4. Функция ассистента

In [ ]:
def assistant(state: MessagesState):
    print(f"\n--- Текущий state ---\n{state['messages'][-1]}\n---") # вывод логов и вывод в консоль последнее сообщение из текущего состояния
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]} # обновленный словарь

'''Метод .invoke() в LangChain — это основной способ выполнения запросов к языковой модели - это часть api

.invoke() — для одиночных запросов (как в вашем коде)

.generate() — для параллельных запросов к модели (например, обработка списка вопросов) '''

### 5. Доп функционал (конвертация валют)

In [ ]:
def convert_currency(amount: float, from_curr: str, to_curr: str) -> float:
    proxies = {"http": "http://43.153.39.191:13001", "https": "http://43.153.39.191:13001"} # здесь нужен хороший proxy
    try:
        response = requests.get(f"https://api.exchangerate-api.com/v4/latest/{from_curr}", proxies=proxies, timeout=10)
        data = response.json() # Парсит JSON-ответ API
        return amount * data["rates"][to_curr]
    except Exception as e:
        raise ValueError(f"Ошибка: {e}")

### 6. Логика маршрутизации

In [ ]:
# реализует логику ветвления и обработки запросов по конвертации валют

def router(state: MessagesState) -> str:

    ''' Определяет, нужно ли обрабатывать запрос как конвертацию валют
     или использовать обычные инструменты (арифметику) '''

    last_msg = state["messages"][-1].content
    if any(word in last_msg.lower() for word in ["usd", "eur", "руб", "конверт"]):
        return "currency"
    return "tools"

def currency_node(state: MessagesState):

    '''Обрабатывает запросы на конвертацию валют, вызывая внешний API '''

    try:
        query = state["messages"][-1].content
        parts = query.split()
        amount, from_curr, to_curr = float(parts[0]), parts[1], parts[3]
        result = convert_currency(amount, from_curr, to_curr)
        return {"messages": [HumanMessage(content=f"Результат: {amount} {from_curr} = {result:.2f} {to_curr}")]}
    except Exception as e:
        return {"messages": [HumanMessage(content=f"Ошибка: {str(e)}")]}

### 7. Построение графа

In [ ]:
builder = StateGraph(MessagesState) # создаем конструктор графа, который будет содержать узлы (ноды) и связи между ними

builder.add_node("assistant", assistant) # конструктор графа, который будет содержать узлы (ноды) и связи между ними
builder.add_node("tools", ToolNode(tools)) # Обрабатывает арифметические операции
builder.add_node("currency", currency_node) # работает с валютами

'''Настройка связей '''

builder.add_edge(START, "assistant") # Стартовая точка
builder.add_conditional_edges("assistant", router, {"tools": "tools", "currency": "currency"}) # ветвление
builder.add_edge("tools", "assistant") # После выполнения tools возвращаем управление агенту
builder.add_edge("currency", "assistant") # После выполнения currency возвращаем управление агенту

final_graph = builder.compile() # создание графа

### 8. Тестирование

In [ ]:
queries = [
    "Сложи 5 и 3, затем раздели результат на 2",
    "Конвертируй 100 USD в EUR",
    "Умножь 10 на 0, затем раздели на 0"
]

for query in queries:
    print(f"\n Запрос: '{query}'")
    result = final_graph.invoke({"messages": [HumanMessage(content=query)]}) # граф получает запрос как первое сообщение.
    print(f"Ответ: {result['messages'][-1].content}")